In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle").sort_values(["user_id", "timestamp"]).reset_index(drop=True)

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]

In [4]:
df.columns

Index(['timestamp', 'user_id', 'content_id', 'content_type_id',
       'task_container_id', 'user_answer', 'answered_correctly',
       'prior_question_elapsed_time', 'prior_question_had_explanation',
       'question_id', 'bundle_id', 'correct_answer', 'part', 'tags', 'tag',
       'type_of'],
      dtype='object')

In [14]:
w_df = df.groupby("user_id").head(10)

In [15]:
w_df["cumcount"] = w_df.groupby("user_id").cumcount()
w_df.groupby(["cumcount", "task_container_id"])["answered_correctly"].describe()

/home/owner/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


count      mean       std  min   25%  50%   75%  \
cumcount task_container_id                                                      
0        0                  39017.0  0.682164  0.465971 -1.0  0.00  1.0  1.00   
         1                    223.0  0.573991  0.563648 -1.0  0.00  1.0  1.00   
         2                     21.0  0.571429  0.507093  0.0  0.00  1.0  1.00   
         3                     10.0  0.500000  0.527046  0.0  0.00  0.5  1.00   
         4                      3.0  0.666667  0.577350  0.0  0.50  1.0  1.00   
         5                      9.0  0.555556  0.527046  0.0  0.00  1.0  1.00   
         6                      2.0  0.000000  0.000000  0.0  0.00  0.0  0.00   
         7                      1.0  0.000000       NaN  0.0  0.00  0.0  0.00   
         8                      1.0  0.000000       NaN  0.0  0.00  0.0  0.00   
         10                     1.0  1.000000       NaN  1.0  1.00  1.0  1.00   
         12                     1.0  1.000000       NaN  1.0  1.00  1.0  1.00   
         13                     2.0  0.500000  0.707107  0.0  0.25  0.5  0.75   
1        0                     88.0  0.511364  0.502735  0.0  0.00  1.0  1.00   
         1                  38838.0  0.526572  0.499351 -1.0  0.00  1.0  1.00   
         2                    278.0  0.629496  0.491216 -1.0  0.00  1.0  1.00   
         3                     38.0  0.684211  0.471069  0.0  0.00  1.0  1.00   
         4                     14.0  0.285714  0.468807  0.0  0.00  0.0  0.75   
         5                     12.0  0.250000  0.452267  0.0  0.00  0.0  0.25   
         6                      6.0  0.333333  0.516398  0.0  0.00  0.0  0.75   
         7                      2.0  0.500000  0.707107  0.0  0.25  0.5  0.75   
         9                      1.0  0.000000       NaN  0.0  0.00  0.0  0.00   
         13                     1.0  0.000000       NaN  0.0  0.00  0.0  0.00   
         14                     2.0  0.000000  0.000000  0.0  0.00  0.0  0.00   
         30                     1.0  0.000000       NaN  0.0  0.00  0.0  0.00   
2        0                     87.0  0.563218  0.498863  0.0  0.00  1.0  1.00   
         1                     19.0  0.526316  0.512989  0.0  0.00  1.0  1.00   
         2                  38769.0  0.432253  0.495447 -1.0  0.00  0.0  1.00   
         3                    319.0  0.664577  0.485998 -1.0  0.00  1.0  1.00   
         4                     45.0  0.577778  0.499495  0.0  0.00  1.0  1.00   
         5                     19.0  0.210526  0.418854  0.0  0.00  0.0  0.00   
         6                      6.0  0.166667  0.408248  0.0  0.00  0.0  0.00   
         7                      5.0  0.600000  0.547723  0.0  0.00  1.0  1.00   
         8                      1.0  1.000000       NaN  1.0  1.00  1.0  1.00   
         10                     1.0  1.000000       NaN  1.0  1.00  1.0  1.00   
         14                     3.0  1.000000  0.000000  1.0  1.00  1.0  1.00   
         17                     1.0  1.000000       NaN  1.0  1.00  1.0  1.00   
         31                     1.0  0.000000       NaN  0.0  0.00  0.0  0.00   
3        0                     35.0  0.771429  0.426043  0.0  1.00  1.0  1.00   
         1                     67.0  0.537313  0.502369  0.0  0.00  1.0  1.00   
         2                     20.0  0.500000  0.512989  0.0  0.00  0.5  1.00   
         3                  38728.0  0.543328  0.498177 -1.0  0.00  1.0  1.00   
         4                    350.0  0.577143  0.511801 -1.0  0.00  1.0  1.00   
         5                     38.0  0.552632  0.503897  0.0  0.00  1.0  1.00   
         6                     15.0  0.266667  0.457738  0.0  0.00  0.0  0.50   
         7                     11.0  0.636364  0.504525  0.0  0.00  1.0  1.00   
         8                      3.0  0.666667  0.577350  0.0  0.50  1.0  1.00   
         9                      1.0  1.000000       NaN  1.0  1.00  1.0  1.00   
         10                     1.0  1.000000       NaN  1.0 

In [18]:

w_df[(w_df["cumcount"]==0) & (w_df["task_container_id"] != 0)].groupby(["task_container_id", "content_id"]).size()

task_container_id  content_id
1                  2              1
                   4              1
                   30             1
                   45             1
                   90             1
                   105            1
                   117            1
                   128            3
                   131            1
                   164            1
                   284            1
                   315            1
                   335            1
                   382            1
                   420            1
                   484            1
                   542            1
                   621            1
                   633            1
                   636            1
                   660            1
                   698            1
                   784            1
                   789            1
                   855            1
                   881            1
                   951            

In [30]:
user_id = w_df[(w_df["task_container_id"] == 1) & (w_df["cumcount"] == 0) & (w_df["content_id"] == 5692)]["user_id"].drop_duplicates().values

In [31]:
w_df[w_df["user_id"].isin(user_id)].to_csv("048_task_container_id/cid1_contentid5692.csv")

In [34]:
df["cumcount"] = df.groupby("user_id").cumcount()

In [37]:
df[df["content_id"]==5716].groupby("task_container_id")["answered_correctly"].describe()

,count,mean,std,min,25%,50%,75%,max
task_container_id,,,,,,,,
0,5.0,0.400000,0.547723,0.0,0.0,0.0,1.0,1.0
1,265.0,0.732075,0.443716,0.0,0.0,1.0,1.0,1.0
2,49.0,0.714286,0.456435,0.0,0.0,1.0,1.0,1.0
3,10.0,0.800000,0.421637,0.0,1.0,1.0,1.0,1.0
4,3.0,0.666667,0.577350,0.0,0.5,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
8159,1.0,1.000000,NaN,1.0,1.0,1.0,1.0,1.0
8479,1.0,1.000000,NaN,1.0,1.0,1.0,1.0,1.0
8738,1.0,1.000000,NaN,1.0,1.0,1.0,1.0,1.0


In [38]:
df[df["content_id"]==5692].groupby("task_container_id")["answered_correctly"].describe()

,count,mean,std,min,25%,50%,75%,max
task_container_id,,,,,,,,
0,271.0,0.712177,0.453586,0.0,0.00,1.0,1.00,1.0
1,56.0,0.660714,0.477752,0.0,0.00,1.0,1.00,1.0
2,8.0,0.875000,0.353553,0.0,1.00,1.0,1.00,1.0
3,2.0,0.500000,0.707107,0.0,0.25,0.5,0.75,1.0
4,6.0,0.666667,0.516398,0.0,0.25,1.0,1.00,1.0
...,...,...,...,...,...,...,...,...
6861,1.0,1.000000,NaN,1.0,1.00,1.0,1.00,1.0
8245,1.0,1.000000,NaN,1.0,1.00,1.0,1.00,1.0
8737,1.0,1.000000,NaN,1.0,1.00,1.0,1.00,1.0


In [33]:
w_df[w_df["user_id"].isin(user_id)].groupby(["cumcount", "content_id"])["answered_correctly"].describe()

count      mean       std  min   25%  50%   75%  max
cumcount content_id                                                      
0        5692         43.0  0.627907  0.489083  0.0  0.00  1.0  1.00  1.0
1        128           1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
         5716         42.0  0.690476  0.467901  0.0  0.00  1.0  1.00  1.0
2        128           2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
         5344         40.0  0.800000  0.405096  0.0  1.00  1.0  1.00  1.0
         7860          1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0
3        128           2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
         6360         38.0  0.684211  0.471069  0.0  0.00  1.0  1.00  1.0
         7860          2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
         7922          1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
4        156           1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
         6410         38.0  0.631579  0.488852  0.0  0.00  1.0  1.00  1.0
         7860          2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
         7922          2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
5        51            1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
         128           1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
         156           2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
         6405         37.0  0.540541  0.505228  0.0  0.00  1.0  1.00  1.0
         7922          2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
6        50            1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
         51            2.0  0.500000  0.707107  0.0  0.25  0.5  0.75  1.0
         156           2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
         5392         37.0  0.459459  0.505228  0.0  0.00  0.0  1.00  1.0
         7860          1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
7        50            2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
         51            2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
         5664         37.0  0.405405  0.497743  0.0  0.00  0.0  1.00  1.0
         7896          1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
         7922          1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
8        50            2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
         156           1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
         4789         37.0  0.486486  0.506712  0.0  0.00  0.0  1.00  1.0
         7863          1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
         7896          2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
9        51            1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
         152           1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
         5213         37.0  0.513514  0.506712  0.0  0.00  1.0  1.00  1.0
         7863          2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
         7896          2.0  0.500000  0.707107  0.0  0.25  0.5  0.75  1.0

In [40]:

user_id = w_df[(w_df["cumcount"]==1) & (w_df["task_container_id"] == 0)]["user_id"].values

In [42]:
w_df[w_df["user_id"].isin(user_id)].to_csv("048_task_container_id/cumcount1-container0.csv")

In [45]:
df["cumcount_under10"] = df["cumcount"]<10
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [46]:
df.groupby(["cumcount_under10", "prior_question_had_explanation"])["answered_correctly"].describe()

count      mean  \
cumcount_under10 prior_question_had_explanation                        
False            False                            586484.0  0.510360   
                 True                            8859193.0  0.671122   
True             False                            323988.0  0.476799   
                 True                              28693.0  0.614749   

                                                      std  min  25%  50%  75%  \
cumcount_under10 prior_question_had_explanation                                 
False            False                           0.499893  0.0  0.0  1.0  1.0   
                 True                            0.469806  0.0  0.0  1.0  1.0   
True             False                           0.499462  0.0  0.0  0.0  1.0   
                 True                            0.486663  0.0  0.0  1.0  1.0   

                                                 max  
cumcount_under10 prior_question_had_explanation       
False            False                           1.0  
                 True                            1.0  
True             False                           1.0  
                 True                            1.0